# 3D Visualization Demo — Optical MEMS PDK

Extrudes the test structures from `test/PDK/full_ruleset_demo.ipynb` into a 3D
OpenSCAD model using **gds3xtrude**. The tech file
(`src/libraries/gds3xtrude_tech.py`) maps each GDS structure layer to its
physical z-position and thickness so that the extruded model reflects the real
process cross-section.

**Pipeline:** gdsfactory `Component` &#8594; GDS file &#8594; KLayout `pya.Layout` &#8594; gds3xtrude `render_scad_to_file` &#8594; `.scad` (open in OpenSCAD)

In [1]:
import sys, os

sys.path.insert(0, "../../libraries")

import gdsfactory as gf
from mcw_custom_optical_mems_pdk import LAYER, DESIGN_RULES as DR, PDK

PDK.activate()

In [2]:
# ---------------------------------------------------------------------------
# Primitive shape generators (same as full_ruleset_demo.ipynb)
# ---------------------------------------------------------------------------

def centered_rect(w, h, layer):
    """Axis-aligned rectangle centered at origin on a single layer."""
    c = gf.Component()
    c.add_polygon(
        [(-w / 2, -h / 2), (w / 2, -h / 2), (w / 2, h / 2), (-w / 2, h / 2)],
        layer=layer,
    )
    return c


def plate_with_holes(w, h, hole_size, hole_pitch, layer):
    """Rectangular plate with a grid of square etch holes (boolean subtraction)."""
    plate = centered_rect(w, h, layer)
    holes = gf.Component()
    margin = hole_pitch / 2 + hole_size / 2
    nx = max(1, int((w - 2 * margin) / hole_pitch) + 1)
    ny = max(1, int((h - 2 * margin) / hole_pitch) + 1)
    for i in range(nx):
        for j in range(ny):
            x = -(nx - 1) * hole_pitch / 2 + i * hole_pitch
            y = -(ny - 1) * hole_pitch / 2 + j * hole_pitch
            ref = holes.add_ref(centered_rect(hole_size, hole_size, layer))
            ref.move((x, y))
    return gf.boolean(
        A=plate, B=holes, operation="not",
        layer1=layer, layer2=layer, layer=layer,
    )


def finger_array(n, length, width, gap, layer):
    """Row of comb fingers extending in +x from x=0, centered in y."""
    c = gf.Component()
    pitch = width + gap
    for i in range(n):
        y = -(n - 1) * pitch / 2 + i * pitch
        c.add_polygon(
            [(0, y - width / 2), (length, y - width / 2),
             (length, y + width / 2), (0, y + width / 2)],
            layer=layer,
        )
    return c

In [3]:
# ---------------------------------------------------------------------------
# Component builders (same as full_ruleset_demo.ipynb)
# ---------------------------------------------------------------------------

def strip_waveguide(length=25.0, width=0.5):
    c = gf.Component()
    for lyr in [LAYER.SI_FULL, LAYER.MASK_SI_FULL_ETCH]:
        c.add_ref(centered_rect(length, width, lyr))
    return c


def rib_waveguide(length=25.0, core_w=0.5, slab_w=5.0, slab_len=20.0):
    c = gf.Component()
    for lyr in [LAYER.SI_FULL, LAYER.MASK_SI_FULL_ETCH]:
        c.add_ref(centered_rect(length, core_w, lyr))
    for lyr in [LAYER.SI_PARTIAL, LAYER.MASK_SI_PARTIAL_ETCH]:
        c.add_ref(centered_rect(slab_len, slab_w, lyr))
    return c


def anchor_pad(metal_s=5.0, anchor_s=10.0, poly_s=11.0, si_s=12.0):
    c = gf.Component()
    psg_s = poly_s - 2 * DR.enc_polysi_of_psg
    hm_s = psg_s + 2 * DR.enc_hard_mask_of_psg_etch
    c.add_ref(centered_rect(si_s, si_s, LAYER.SI_FULL))
    c.add_ref(centered_rect(si_s, si_s, LAYER.OXIDE_LTO))
    c.add_ref(centered_rect(psg_s, psg_s, LAYER.OXIDE_PSG))
    c.add_ref(centered_rect(anchor_s, anchor_s, LAYER.POLY_ANCHOR))
    c.add_ref(centered_rect(poly_s, poly_s, LAYER.POLY_TOP))
    c.add_ref(centered_rect(metal_s, metal_s, LAYER.METAL))
    c.add_ref(centered_rect(anchor_s, anchor_s, LAYER.MASK_ANCHOR_ETCH))
    c.add_ref(centered_rect(psg_s, psg_s, LAYER.MASK_PSG_ETCH))
    c.add_ref(centered_rect(metal_s, metal_s, LAYER.MASK_METAL_LIFTOFF))
    c.add_ref(centered_rect(hm_s, hm_s, LAYER.HARD_MASK))
    c.add_ref(centered_rect(metal_s, metal_s, LAYER.METAL_LIFTOFF_AUX))
    return c


def mems_geometry(layer, plate_w=24.0, plate_h=12.0,
                  hole_size=2.5, hole_pitch=8.0,
                  n_fingers=4, finger_l=7.0, finger_w=0.5, finger_gap=0.5,
                  beam_w=1.0, beam_l=15.0):
    c = gf.Component()
    c.add_ref(plate_with_holes(plate_w, plate_h, hole_size, hole_pitch, layer))
    ref = c.add_ref(finger_array(n_fingers, finger_l, finger_w, finger_gap, layer))
    ref.move((plate_w / 2, 0))
    c.add_polygon(
        [(-plate_w / 2 - beam_l, -beam_w / 2), (-plate_w / 2, -beam_w / 2),
         (-plate_w / 2, beam_w / 2), (-plate_w / 2 - beam_l, beam_w / 2)],
        layer=layer,
    )
    return c


def mems_actuator():
    c = gf.Component()
    plate_w, plate_h = 24.0, 12.0
    beam_l, finger_l = 15.0, 7.0
    c.add_ref(mems_geometry(LAYER.POLY_MEMS))
    c.add_ref(mems_geometry(LAYER.MASK_POLYSI_ETCH))
    psg_margin = 1.0
    total_w = plate_w + beam_l + finger_l + 2 * psg_margin
    total_h = plate_h + 2 * psg_margin
    cx = (finger_l - beam_l) / 2
    ref_psg = c.add_ref(centered_rect(total_w, total_h, LAYER.OXIDE_PSG))
    ref_psg.move((cx, 0))
    release_margin = 4.0
    ref_rel = c.add_ref(centered_rect(
        total_w + 2 * release_margin,
        total_h + 2 * release_margin,
        LAYER.RELEASE,
    ))
    ref_rel.move((cx, 0))
    return c

In [4]:
# ---------------------------------------------------------------------------
# Assemble all structures and write GDS
# ---------------------------------------------------------------------------

def full_ruleset_demo():
    """Assemble all structures into a single component."""
    demo = gf.Component("full_ruleset_demo")

    # Background chip area (substrate + buried oxide)
    demo.add_ref(centered_rect(140.0, 45.0, LAYER.SI_SUBSTRATE))
    demo.add_ref(centered_rect(140.0, 45.0, LAYER.OXIDE_SOI))

    # Photonic waveguides (left region)
    ref_strip = demo.add_ref(strip_waveguide())
    ref_strip.move((-42, -4))
    ref_rib = demo.add_ref(rib_waveguide())
    ref_rib.move((-42, 6))

    # Anchor pad with metal contact (center)
    demo.add_ref(anchor_pad())

    # Released MEMS actuator (right region)
    ref_mems = demo.add_ref(mems_actuator())
    ref_mems.move((38, 0))

    return demo


demo = full_ruleset_demo()

# Write GDS for gds3xtrude to consume
gds_path = os.path.join(os.path.dirname(os.path.abspath(".")), "3D", "full_ruleset_demo_3d.gds")
demo.write_gds(gds_path)
print(f"GDS written to: {gds_path}")

GDS written to: /Users/neekon/git/mcw/mcw-mems-photonic-switch/src/test/3D/full_ruleset_demo_3d.gds


In [5]:
# ---------------------------------------------------------------------------
# 3D extrusion with gds3xtrude
# ---------------------------------------------------------------------------
# Only structure layers (1-10) are extruded — mask and dummy layers have no
# physical thickness and are omitted by the tech file.

import klayout.db as pya
from gds3xtrude.openscad import render_scad_to_file

# Load the GDS into a KLayout layout object
layout = pya.Layout()
layout.read(gds_path)
top_cell = layout.top_cell()

# Resolve path to gds3xtrude tech file
tech_path = os.path.abspath("../../libraries/gds3xtrude_tech.py")

# Render to OpenSCAD (.scad)
scad_path = gds_path.replace(".gds", ".scad")
render_scad_to_file(layout, top_cell, tech_path, scad_path,
                    centered=True, scale_factor=1.0)

print(f"OpenSCAD file written to: {scad_path}")
print(f"Open in OpenSCAD to view the 3D model: openscad {scad_path}")

OpenSCAD file written to: /Users/neekon/git/mcw/mcw-mems-photonic-switch/src/test/3D/full_ruleset_demo_3d.scad
Open in OpenSCAD to view the 3D model: openscad /Users/neekon/git/mcw/mcw-mems-photonic-switch/src/test/3D/full_ruleset_demo_3d.scad


/Users/neekon/git/mcw/mcw-mems-photonic-switch/switch-env/lib/python3.11/site-packages/solid/solidpython.py:26: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
